<a href="https://colab.research.google.com/github/alphakilo11/Python/blob/main/wip_mtg_2210081135.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/MagicTheGathering/mtg-sdk-python
!pip install mtgsdk
from mtgsdk import Card
from mtgsdk import Set

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mtgsdk: filename=mtgsdk-1.3.1-py3-none-any.whl size=13920 sha256=b5d409a764bd1bad029d73b86f0ab02612992927cd6e8e7fe8ae0e3049564420
  Stored in directory: /root/.cache/pip/wheels/13/b4/0d/d22b3e61b8ee5b6377b891165a1338196c11fc133673fe002b
Successfully built mtgsdk


In [2]:
def cardnames(cards):
  """print list of card names"""
  return list(i.name for i in cards)

def create_decklist(list_of_cards, setname, number_of_each_card):
  """ creates a decklist in Forge format (*.dck)"""
  decklist = '[metadata]\nName=' + setname +'\n[Main]\n'
  for card in list_of_cards:
    decklist += str(number_of_each_card) + ' ' + card.name + '|' + setname + '\n'
  return decklist

In [ ]:
#example
cards = Card.where(name='"Hex"').where(pageSize=1).all()
print(cardnames(cards)) # print list of card names

from pprint import pprint
pprint(vars(cards[0])) # print all properties of the first card

In [17]:
#FIX currently the random decks contain an uneven number of cards
#ENHANCE the program should determine the average CMC of each deck and adjust the number of lands accordingly
import random
setname ='GTC'
decksize = 36 # without additional lands
#cards = Card.where(set=setname).all()
number_of_decks = len(cards) // decksize
decks = {}
decknames = []
for i in range(number_of_decks):
  name = setname + '_' + str(i) + '.dck'
  decknames.append(name)
  decks[name] = []
print(decks)

for card in cards:
  decks[random.choice(decknames)].append(card)

for name in decknames:
  filename = '/content/drive/MyDrive/Brettspiele&Co/Magic the Gathering/Decklists/Autogen/' + name
  with open(filename, 'x') as file:
    file.write(create_decklist(decks[name], setname, 1))


{'GTC_0.dck': [], 'GTC_1.dck': [], 'GTC_2.dck': [], 'GTC_3.dck': [], 'GTC_4.dck': [], 'GTC_5.dck': []}


In [ ]:
# setlists are ordered by color with lands last (testet with KTK and 2ED)
setname ='2ED'
number_of_cards = 36

seite = 1
while True:
  cards = Card.where(page=seite).where(pageSize=number_of_cards).where(set=setname).all()
  if len(cards) != number_of_cards:
    print('Number of cards provided is unequeal', number_of_cards, '. Aborting...')
    break
  filename = '/content/drive/MyDrive/Brettspiele&Co/Magic the Gathering/Decklists/Autogen/' + setname + '_' + str(seite) + '.dck'
  with open(filename, 'x') as file:
    file.write(create_decklist(cards, setname, 1))
  seite += 1

FileExistsError: ignored

In [ ]:
boosterpack = Set.generate_booster('ktk')
print(list(i.name for i in boosterpack))

['Mantis Rider', 'Winterflame', 'Pine Walker', 'Seeker of the Way', 'Highland Game', 'Cancel', 'Plains', 'Temur Banner', 'Shatter', 'Longshot Squad', 'Weave Fate', "Archers' Parapet", 'Debilitating Injury', 'Plains']


In [ ]:
# possible phase class characteristics: phases end when all players pass with stack empty
# possible both chars: END: emtpy mana, "until end of" effects expire BEGIN: "until" effects expire, "at the beginning of" triggers
#HeadsUp: “until end of combat” last until end of combat PHASE
phases = "beginning", "precombat main", "combat", "postcombat main", "ending" 
beginning_steps = "untap", "upkeep", "draw"
combat_steps = "beginning of combat", "declare attackers", "declare blockers", "combat damage", "end of combat"
ending_steps = "end", "cleanup"

def turn(player, number):
    print(f"{player}`s Turn {number}.")
    print("Beginning: Untap")
    print("Trigger: 'At Beginning of Turn' and 'At Beginning of Untap Step' (go on the stack at beginning of Upkeep)")
    print("Phasing")
    print(f"{player} untaps.")

    print("Beginning: Upkeep")
    print("Trigger: 'At Beginning of Upkeep'")
    priority(player)

    print("Beginning: Draw")
    print("draw(library, 1)")
    print("Trigger: 'At Beginning of Draw Step'")
    priority(player)


    print("Precombat Main")
    priority(player)

    print("Combat: Beginning of Combat")
    priority(player)

    print("Combat: Declare Attackers")
    priority(player)

    print("Combat: Declare Blockers")
    priority(player)

    print("Combat: Combat Damage")
    priority(player)

    print("Combat: End of Combat")
    priority(player)

    print("Postcombat Main")
    priority(player)

    print("Ending: End")
    print("Trigger: 'At Beginning of your end step'")

    print("Ending: Cleanup")

def priority(player):
    """When a player can do something, eg cast spells"""
    print("checking for state-based effects")
    input("Make your move!")


In [ ]:
#TODO work with JSON files
#BUG drawn cards are added as lists inside a list
#BUG sorting the hand doesn't work properly 

import random

#the decklist as dictionary (manually compiled)
deck1 = {
"Esper Sentinel":4 ,
"Giver of Runes":4 ,
"Memnite":4 ,
"Ornithopter":4 ,
"Puresteel Paladin":4 ,
"Stoneforge Mystic":4 ,
"Steelshaper's Gift":1 ,
"Colossus Hammer":4 ,
"Shadowspear":1 ,
"Springleaf Drum":3 ,
"Sigarda's Aid":4 ,
"Horizon Canopy":2 ,
"Inkmoth Nexus":4 ,
"Plains":11,
"Silent Clearing":2 ,
"Urza's Saga":4
}

# creating and shuffling the library
library = []
for key in deck1:
    for i in range(deck1[key]):
        library.append(key)
        #library.append(Card.where(name=key).all()) # it takes 88 s to fetch a deck of 60 cards
    #print(deck1[i], i)
random.shuffle(library)

"""Draw function
for now it just removes the cards from the library and returns the drawn cards.
Maybe they should be added to hand right away, but I think some cards have conditions before being added to the hand"""
def draw(library, number):
    if number <= len(library):
        drawn_cards = []
        for i in range(number):
            drawn_cards.append(library.pop(0))
        return drawn_cards
    else:
        global gameover
        gameover = 1
        print("You tried to draw more cards than your library contains and therefore loose the game.")


#draw starting hand
hand = []
hand.append(draw(library, 7))

gameover = 0
playername = "Alex"

while gameover == 0:
    print(f"{playername}´s library contains {len(library)} cards. Hand: {sorted(hand)}") #TODO sort hand by CMC and type
    useraction = input("What do you want to do? draw - d, concede - c")

    if useraction == "c":
        break

    if useraction == "d":
        try:
            carddraw = []
            carddraw = draw(library, int(input("How many cards do you want to draw?")))
            hand.append(carddraw)
            if carddraw != None:
                print(f"{playername} draws: {carddraw}")
                        
        except ValueError:
            print("Please enter an integer.")

print("Game ended.")

Alex´s library contains 53 cards. Hand: [['Inkmoth Nexus', 'Plains', "Sigarda's Aid", 'Horizon Canopy', 'Plains', 'Plains', "Sigarda's Aid"]]
What do you want to do? draw - d, concede - cd
How many cards do you want to draw?3
Alex draws: ['Esper Sentinel', 'Silent Clearing', 'Silent Clearing']
Alex´s library contains 50 cards. Hand: [['Esper Sentinel', 'Silent Clearing', 'Silent Clearing'], ['Inkmoth Nexus', 'Plains', "Sigarda's Aid", 'Horizon Canopy', 'Plains', 'Plains', "Sigarda's Aid"]]
What do you want to do? draw - d, concede - cc
Game ended.
